# 1) Introduction

L’objectif du notebook est de pré traiter les données issues de l’outil d’export de données de Stack Overflow-
"stackexchange explorer", qui recense un grand nombre de données authentiques de la plateforme afin de détecter
les sujets et générer des tags.

## 1.1. Étapes de prétraitement

**1) Suppression du bruit**

1. Suppression du formatage HTML
2. Suppression des contractions
3. La correction orthographique
4. Mettre en minuscule le texte

**2) Suppression des caractères simples**

1. Suppression de la ponctuation, des caractères spéciaux et des nombres
2. Suppression d'un seul caractère (facultatif et spécifique)

**3) Suppression de StopWords**

1. Suppression du mot le plus fréquent
2. Suppression d'un certain type de mot (facultatif et spécifique)

**4) Stemming / Lemmatisation**

1. Stemming
2. Lemmatisation


Ce pré-processus est utilisé pour effectuer une simple détection de sujet (LDA, NMF, etc.) ou une classification,
des informations nécessaires à certaines analyses peuvent être perdues.

 

1.1.1 Vocabulaire
Si vous êtes nouveau dans la PNL, voici une petite liste de concepts utilisés dans ce cahier.

Tokenize : "Processus de conversion d'une chaîne en une liste de sous-chaînes, appelées tokens."

Normalisation du texte : "Processus de transformation d'un texte en une seule forme canonique qu'il n'aurait
peut-être pas eu auparavant (par exemple, mettre en minuscule le texte, supprimer les contractions, correction
orthographique, stemming / lemmatisation, etc.). La normalisation du texte nécessite de savoir quel type de texte
doit être normalisée et comment elle doit être traitée par la suite ; il n’existe pas de procédure de
normalisation universelle. "

Suppression du bruit : "Processus de suppression de tout élément susceptible interferer avec votre analyse
(par exemple, suppression du code HTML, mettre en minuscule le texte, suppression de la ponctuation / du caractère
spécial, etc.)

Stemming: "Processus de réduction des mots à leur racine , base ou forme de racine - généralement une forme de
mot écrit ("fishing", "fished", and "fisher" to the stem "fish")."

Lemmatisation : "Processus de regroupement des formes fléchies d'un mot afin qu'elles puissent être analysées
comme un seul élément, identifié par le lemme du mot, ou par la forme du dictionnaire (ie : "walking" to "walk",
 "better" to "good")."

StopWord : "Mots qui sont filtrés avant ou après le traitement des données en langage naturel (texte). Les mots
d'arrêt font généralement référence aux mots les plus courants dans une langue (des mots comme "The","a", etc. en
anglais)."

# 2) Libraries and Dataset 

In [1]:
! pip install bs4
! pip install contractions
! pip install autocorrect 

In [2]:
# generic librairies
import pandas as pd

# Text librairies
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import contractions
from autocorrect import Speller

In [3]:
# https://numpy.org/devdocs/user/basics.types.html

dtypes_questions = {'Id':'int32', 'Score': 'int16', 'Title': 'string', 'Body': 'string', 'Tags': 'string'}

In [4]:
%%time

nrows=1000

df_questions_python = pd.read_csv('input/QueryResults python score more than 20.csv',
                           usecols=dtypes_questions.keys(),
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_questions_r = pd.read_csv('input/QueryResults r score more than 20.csv',
                           usecols=['Id', 'Score', 'Title', 'Body', 'Tags'],
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_questions = pd.concat([df_questions_python,df_questions_r])
df_questions['Title_raw']=df_questions['Title']
df_questions = df_questions.sample(frac=1).reset_index(drop=True)

CPU times: user 34.3 ms, sys: 1.6 ms, total: 35.9 ms
Wall time: 43.3 ms


In [5]:
df_questions[['Title', 'Body', 'Tags']] = df_questions[[
    'Title', 'Body','Tags'
]].applymap(lambda x: str(x).encode("utf-8", errors='surrogatepass').decode(
    "ISO-8859-1", errors='surrogatepass'))

In [6]:
df_questions.dtypes

Id            int32
Title        object
Body         object
Tags         object
Score         int16
Title_raw    string
dtype: object

In [7]:
# Remove all questions that have a < 30 score
df_questions = df_questions[df_questions["Score"] >= 20]

In [8]:
spell = Speller()
token = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'
stop_words = set(stopwords.words("english"))
adjective_tag_list = {'JJ', 'JJR', 'JJS', 'RBR', 'RBS'}  # List of Adjective's tag from nltk package


**Tag list**  
List of tag use in the tagger (pos_tag function) from NLTK:
https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [9]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         2000 non-null   int32 
 1   Title      2000 non-null   object
 2   Body       2000 non-null   object
 3   Tags       2000 non-null   object
 4   Score      2000 non-null   int16 
 5   Title_raw  2000 non-null   string
dtypes: int16(1), int32(1), object(3), string(1)
memory usage: 89.8+ KB


In [10]:
df_questions['qTitle'] = df_questions['Tags'].apply(lambda x : len((x).split(" ")))

# 3) Suppression du bruit

La suppression du bruit consiste à supprimer tout ce qui peut interférer avec votre analyse de texte. C'est comme l'étape de nettoyage des données pour un projet ML classique.

## 3.1. Suppression du code HTML

In [11]:
df_questions['Body'][10]

"<p>I need to know if a variable in Python is a string or a dict.  Is there anything wrong with the following code?</p>\n\n<pre><code>if type(x) == type(str()):\n    do_something_with_a_string(x)\nelif type(x) == type(dict()):\n    do_somethting_with_a_dict(x)\nelse:\n    raise ValueError\n</code></pre>\n\n<p><strong>Update</strong>: I accepted avisser's answer (though I will change my mind if someone explains why <code>isinstance</code> is preferred over <code>type(x) is</code>).</p>\n\n<p>But thanks to nakedfanatic for reminding me that it's often cleaner to use a dict (as a case statement) than an if/elif/else series.</p>\n\n<p>Let me elaborate on my use case.  If a variable is a string, I need to put it in a list.  If it's a dict, I need a list of the unique values.  Here's what I came up with:</p>\n\n<pre><code>def value_list(x):\n    cases = {str: lambda t: [t],\n             dict: lambda t: list(set(t.values()))}\n    try:\n        return cases[type(x)](x)\n    except KeyError:\

In [12]:
%%time

# Analyser la question et le titre puis renvoyer uniquement le texte
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())


CPU times: user 744 ms, sys: 4.49 ms, total: 749 ms
Wall time: 749 ms


BeautifulSoup nous permet de supprimer efficacement la plupart du code html mais pas tout.

In [13]:
df_questions['Body'][10]

"I need to know if a variable in Python is a string or a dict.  Is there anything wrong with the following code?\nif type(x) == type(str()):\n    do_something_with_a_string(x)\nelif type(x) == type(dict()):\n    do_somethting_with_a_dict(x)\nelse:\n    raise ValueError\n\nUpdate: I accepted avisser's answer (though I will change my mind if someone explains why isinstance is preferred over type(x) is).\nBut thanks to nakedfanatic for reminding me that it's often cleaner to use a dict (as a case statement) than an if/elif/else series.\nLet me elaborate on my use case.  If a variable is a string, I need to put it in a list.  If it's a dict, I need a list of the unique values.  Here's what I came up with:\ndef value_list(x):\n    cases = {str: lambda t: [t],\n             dict: lambda t: list(set(t.values()))}\n    try:\n        return cases[type(x)](x)\n    except KeyError:\n        return None\n\nIf isinstance is preferred, how would you write this value_list() function?\n"

Nous devons donc supprimer le reste ici.

In [14]:
def clean_text(text):
    """ mettre la déf de la fonction"""
    text = re.sub(r"\'", "'", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r"\n", " ", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = re.sub(r"\xa0", " ", text) # match all literal non-breakable space pattern then replace them by a single whitespace
    text = re.sub('\s+', ' ', text) # match all one or more whitespace then replace them by a single whitespace
    text = text.strip(' ')
    return text


In [15]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: clean_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: clean_text(x))

CPU times: user 75 ms, sys: 990 µs, total: 76 ms
Wall time: 75.6 ms


In [16]:
df_questions['Body'][10]

"I need to know if a variable in Python is a string or a dict. Is there anything wrong with the following code? if type(x) == type(str()): do_something_with_a_string(x) elif type(x) == type(dict()): do_somethting_with_a_dict(x) else: raise ValueError Update: I accepted avisser's answer (though I will change my mind if someone explains why isinstance is preferred over type(x) is). But thanks to nakedfanatic for reminding me that it's often cleaner to use a dict (as a case statement) than an if/elif/else series. Let me elaborate on my use case. If a variable is a string, I need to put it in a list. If it's a dict, I need a list of the unique values. Here's what I came up with: def value_list(x): cases = {str: lambda t: [t], dict: lambda t: list(set(t.values()))} try: return cases[type(x)](x) except KeyError: return None If isinstance is preferred, how would you write this value_list() function?"

Nous devons aussi traiter la colonne des tags


## 3.2. Suppression des contractions

In [17]:
def expand_contractions(text):
    """développer les mots raccourcis, e.g. 'don't' to 'do not'"""
    text = contractions.fix(text)
    return text

In [18]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: expand_contractions(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: expand_contractions(x))

CPU times: user 89.6 ms, sys: 621 µs, total: 90.2 ms
Wall time: 89.7 ms


In [19]:
df_questions['Body'][10]

"I need to know if a variable in Python is a string or a dict. Is there anything wrong with the following code? if type(x) == type(str()): do_something_with_a_string(x) elif type(x) == type(dict()): do_somethting_with_a_dict(x) else: raise ValueError Update: I accepted avisser's answer (though I will change my mind if someone explains why isinstance is preferred over type(x) is). But thanks to nakedfanatic for reminding me that it is often cleaner to use a dict (as a case statement) than an if/elif/else series. Let me elaborate on my use case. If a variable is a string, I need to put it in a list. If it is a dict, I need a list of the unique values. here is what I came up with: def value_list(x): cases = {str: lambda t: [t], dict: lambda t: list(set(t.values()))} try: return cases[type(x)](x) except KeyError: return None If isinstance is preferred, how would you write this value_list() function?"

## 3.3. La correction orthographique

Pour 1000 entrées cette correction prends 10 minutes

In [20]:
def autocorrect(text):
    words_id = token.tokenize(text)
    words_correct = [spell(i) for i in words_id]
    return ' '.join(map(str, words_correct)) # Return the text untokenize

#df_questions['Title'] = df_questions['Title'].apply(lambda x: autocorrect(x))
#df_questions['Body'] = df_questions['Body'].apply(lambda x: autocorrect(x))

## 3.4. Mettre en minuscule le texte

Je choisis d'abaisser le texte après le paquet de contractions car celui-ci peut remettre des lettres majuscules lors de la suppression des contractions. La mise en minuscule du texte est une étape classique et utile de la suppression du bruit ou de la normalisation du texte car elle réduit le vocabulaire, normalise le texte et ne coûte presque rien.

In [21]:
%%time

df_questions['Title'] = df_questions['Title'].str.lower()
df_questions['Body'] = df_questions['Body'].str.lower()
df_questions['Tags'] = df_questions['Tags'].str.lower()

CPU times: user 4.09 ms, sys: 35 µs, total: 4.13 ms
Wall time: 3.77 ms


In [22]:
df_questions['Body'][10]

"i need to know if a variable in python is a string or a dict. is there anything wrong with the following code? if type(x) == type(str()): do_something_with_a_string(x) elif type(x) == type(dict()): do_somethting_with_a_dict(x) else: raise valueerror update: i accepted avisser's answer (though i will change my mind if someone explains why isinstance is preferred over type(x) is). but thanks to nakedfanatic for reminding me that it is often cleaner to use a dict (as a case statement) than an if/elif/else series. let me elaborate on my use case. if a variable is a string, i need to put it in a list. if it is a dict, i need a list of the unique values. here is what i came up with: def value_list(x): cases = {str: lambda t: [t], dict: lambda t: list(set(t.values()))} try: return cases[type(x)](x) except keyerror: return none if isinstance is preferred, how would you write this value_list() function?"

# 4) Suppression des caractères

## 4.1. Suppression de la ponctuation, des caractères spéciaux et des nombres

TOUS les caractères non alphabétiques ont été supprimés (y compris la ponctuation, les nombres et les caractères spéciaux). Ainsi, je ne considère pas les mots importants qui peuvent contenir des caractères spéciaux (comme "C #" en programmation).

In [23]:
def remove_punctuation_and_number(text):
    """remove all punctuation and number"""
    return text.translate(str.maketrans(" ", " ", charac)) 

def remove_non_alphabetical_character(text):
    """remove all non-alphabetical character"""
    text = re.sub("[^a-z]+", " ", text) # remove all non-alphabetical character
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    return text

In [24]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_punctuation_and_number(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_punctuation_and_number(x))

CPU times: user 19.4 ms, sys: 0 ns, total: 19.4 ms
Wall time: 19.1 ms


In [25]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_non_alphabetical_character(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_non_alphabetical_character(x))

CPU times: user 110 ms, sys: 281 µs, total: 110 ms
Wall time: 109 ms


In [26]:
df_questions['Body'][10]

'i need to know if a variable in python is a string or a dict is there anything wrong with the following code if typex typestr dosomethingwithastringx elif typex typedict dosomethtingwithadictx else raise valueerror update i accepted avissers answer though i will change my mind if someone explains why isinstance is preferred over typex is but thanks to nakedfanatic for reminding me that it is often cleaner to use a dict as a case statement than an ifelifelse series let me elaborate on my use case if a variable is a string i need to put it in a list if it is a dict i need a list of the unique values here is what i came up with def valuelistx cases str lambda t t dict lambda t listsettvalues try return casestypexx except keyerror return none if isinstance is preferred how would you write this valuelist function'

## 4.2. Suppression de la présence d'un seul caractère


Je choisis de supprimer un seul caractère car lorsque nous faisons de la programmation, nous utilisons souvent un seul caractère alphabétique comme nom de variable ("x", "y", "z", etc.). Et j'ai observé que lorsque j'ai essayé de détecter des sujets sans les supprimer, j'ai trouvé beaucoup de sujets avec eux! Et même un sujet que je pourrais nommer "Nom de variable" ...

In [27]:
def remove_single_letter(text):
    """remove single alphabetical character"""
    text = re.sub(r"\b\w\b", "", text) # remove all single letter
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    text = text.strip(" ")
    return text

In [28]:
%%time

#df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_single_letter(x))
#df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_single_letter(x))

# nous ne pouvons pas supprimer les single letters car nous voulons garder la lettre R !!

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.58 µs


In [29]:
df_questions['Body'][10]

'i need to know if a variable in python is a string or a dict is there anything wrong with the following code if typex typestr dosomethingwithastringx elif typex typedict dosomethtingwithadictx else raise valueerror update i accepted avissers answer though i will change my mind if someone explains why isinstance is preferred over typex is but thanks to nakedfanatic for reminding me that it is often cleaner to use a dict as a case statement than an ifelifelse series let me elaborate on my use case if a variable is a string i need to put it in a list if it is a dict i need a list of the unique values here is what i came up with def valuelistx cases str lambda t t dict lambda t listsettvalues try return casestypexx except keyerror return none if isinstance is preferred how would you write this valuelist function'

# 5) Suppression des stopwords

## 5.1. Removing most frequent words

Supprimer les mots les plus fréquents est une étape classique de la NLP. Les mots les plus fréquents n'ajoutent pas beaucoup d'informations dans la plupart des cas (puisqu'ils sont dans presque toutes les phrases). En les supprimant, vous créez plus d'"espace" pour les autres mots qui peuvent avoir des informations plus utiles.
Vous pouvez utiliser des listes prédéfinies à partir de bibliothèques telles que SciKit-Learn, NLTK et autres. Mais sachez que ces listes peuvent être plus problématiques qu'utiles (en particulier la liste scikit-learn, voir [Stop Word Lists in Free Open-source Software Packages](https://www.aclweb.org/anthology/W18-2502.pdf) pour plus d'informations).

In [30]:
def remove_stopwords(text):
    """remove common words in english by using nltk.corpus's list"""
    words_idx = token.tokenize(text)
    filtered = [i for i in words_idx if not i in stop_words]
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [31]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_stopwords(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_stopwords(x))

CPU times: user 284 ms, sys: 0 ns, total: 284 ms
Wall time: 283 ms


In [32]:
df_questions['Body'][10]

'need know variable python string dict anything wrong following code typex typestr dosomethingwithastringx elif typex typedict dosomethtingwithadictx else raise valueerror update accepted avissers answer though change mind someone explains isinstance preferred typex thanks nakedfanatic reminding often cleaner use dict case statement ifelifelse series let elaborate use case variable string need put list dict need list unique values came def valuelistx cases str lambda dict lambda listsettvalues try return casestypexx except keyerror return none isinstance preferred would write valuelist function'

## 5.2. Suppression d'adjectives

Je choisis de supprimer les adjectifs en plus de la liste NLTK. Pourquoi ? Tout simplement parce que lorsque j'ai d'abord essayé de faire une détection de sujet dans un cahier suivant celui-ci et cela améliore ma détection de sujet. Je pensais aussi que les adjectifs n'ajouteraient aucune information utile. En même temps, je pourrais aussi supprimer des verbes avec le même raisonnement. Mais je ne l'ai pas fait parce que l'ensemble de données StackOverflow concerne la programmation. Et en programmation, nous avons beaucoup de verbes, ou de mots qui peuvent être interprétés comme un verbe, qui peuvent être importants ("return", "get", "request", "replace", etc.).

In [33]:

# noinspection PyTypeChecker
def remove_by_tag(text, undesired_tag):
    """remove all words by using ntk tag (adjectives, verbs, etc.)"""
    words_idx = token.tokenize(text) # Tokenize each words
    words_tagged = nltk.pos_tag(tokens=words_idx, tagset=None, lang='eng') # Tag each words and return a list of tuples (e.g. ("have", "VB"))
    filtered = [i[0] for i in words_tagged if i[1] not in undesired_tag] # Select all words that don't have the undesired tags
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [34]:
%%time
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))

CPU times: user 5.17 s, sys: 30.9 ms, total: 5.2 s
Wall time: 5.21 s


In [35]:
df_questions['Body'][10]

'need know python string dict anything following code typestr dosomethingwithastringx elif typex typedict dosomethtingwithadictx else raise valueerror accepted avissers answer though change mind someone explains isinstance thanks reminding often dict case statement series let elaborate use case string need put dict need list values came def cases str dict lambda listsettvalues try return casestypexx except return none isinstance preferred would write valuelist function'

# 6) Stemming / Lemmatisation

Le Stemming et la Lemmatisation sont des opérations qui :
- peuvent améliorer votre temps de calcul en réduisant votre vocabulaire
- aider à généraliser plus facilement en regroupant les mots (ex: "suis", "sont", "être", etc. seront transformés en "être" pour la lemmatisation)

## 6.1. Stemming

Je n'ai pas choisi d'utiliser le stemming ici, mais l'on doit toujours envisager cette alternative car elle est beaucoup moins coûteuse.

Le stemming est le processus de réduction des mots fléchis à leur racine mot, base ou forme de racine - généralement une forme de mot écrit ("fishing", "fished", and "fisher" à la racine "fish"). Il fonctionne généralement en supprimant l'affixe d'un mot. Un affixe peut être un suffixe ou un préfixe (par exemple «-ed», «-ing», etc.). C'est simple mais ne fonctionnera pas lorsque le mot est "irrégulier" ("ran" et "run"). C'est une opération plus simple que la lemmatisation, qui peut suffire dans certains cas, mais peut faire trop d'erreurs dans d'autres cas.

In [36]:
words = ["program", "programs", "programer", "programing", "programers"]
  
for w in words:
    print(w, " : ", stemmer.stem(w))

program  :  program
programs  :  program
programer  :  program
programing  :  program
programers  :  program


In [37]:
def stem_text(text):
    """Stem the text"""
    words_idx = nltk.word_tokenize(text) # tokenize the text then return a list of tuple (token, nltk_tag)
    stemmed_text = []
    for word in words_idx:
        stemmed_text.append(stemmer.stem(word)) # Stem each words
    return " ".join(stemmed_text) # Return the text untokenize

In [38]:
# %%time

# df_questions['Title'] = df_questions['Title'].apply(lambda x: stem_text(x))
# df_questions['Body'] = df_questions['Body'].apply(lambda x: stem_text(x))

## 6.2. Lemmatisation

Comme dit au début, la lemmatisation est le processus de remplacement d'un mot par son lemma (forme canonique ou forme dictionnaire). Mais dans certains cas, un lemmatiseur peut ne pas être en mesure de trouver la bonne racine si vous ne précisez pas le type de mot comme vous pouvez le voir ci-dessous.

In [39]:
print(lemmatizer.lemmatize("stripes", "v"))
print(lemmatizer.lemmatize("stripes", "n"))  
print(lemmatizer.lemmatize("are"))
print(lemmatizer.lemmatize("are", "v"))

strip
stripe
are
be


Une façon de contourner ce problème consiste à utiliser un marqueur et à passer le type de mot dans la fonction lemmatise. MAIS c'est vraiment coûteux. La mise en tige ou une simple lemmatisation à cet égard est bien plus efficace.

In [40]:
def lemmatize_text(text):
    """Lemmatize the text by using tag """
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))  # tokenize the text then return a list of tuple (token, nltk_tag)
    lemmatise_text = []
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatise_text.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    return " ".join(lemmatise_text) # Return the text untokenize

In [41]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: lemmatize_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: lemmatize_text(x))

CPU times: user 5.09 s, sys: 27.4 ms, total: 5.11 s
Wall time: 5.12 s


In [42]:
df_questions['Body'][10]

'need know python string dict anything follow code typestr dosomethingwithastringx elif typex typedict dosomethtingwithadictx else raise valueerror accepted avissers answer though change mind someone explain isinstance thanks remind often dict case statement series let elaborate use case string need put dict need list value come def case str dict lambda listsettvalues try return casestypexx except return none isinstance prefer would write valuelist function'

# 7) Feature engineering

L'utilisation du titre et du corps en même temps donne de bien meilleurs résultats pour la détection des sujets.

In [43]:
df_questions['Text'] = df_questions['Title'] + ' ' + df_questions['Body']


# 8) Dataframe avec les Tags

In [44]:
df_questions['Tags'][10]

'<python><types><typechecking>'

In [45]:
def clean_split_tag(text):
    """ mettre la déf de la fonction"""
    text = re.sub(r"<", "", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r">", ",", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = text.strip(',')
    text = text.split(',')
    #text = re.sub(r",", " ", text)
    #text = text.rstrip('>')
    return text

In [46]:
%%time

df_questions['Tags'] = df_questions['Tags'].apply(lambda x: clean_split_tag(x))

CPU times: user 14.1 ms, sys: 159 µs, total: 14.2 ms
Wall time: 13.7 ms


In [47]:
tags_list = df_questions[['Id','Tags']].explode('Tags')
print(tags_list.head(10))

def untokenize(words_id):
    return ' '.join(map(str, words_id)) # Return the text untokenize

df_questions['Tags'] = df_questions['Tags'].apply(lambda x: untokenize(x))

         Id                    Tags
0  10679131                       r
0  10679131  multidimensional-array
1   1169714                       r
1   1169714   programming-languages
1   1169714                       s
2   8096313                       r
2   8096313                 package
3    526457                  python
3    526457                  django
4  10437442                       r


# 9) Exportation des données

In [48]:
print(df_questions.head(10),'\n \n',tags_list.head(10))
#df_questions.drop(['Tags'], axis=1).to_csv('df_questions_fullclean.csv', encoding='utf-8')
df_questions.to_csv('df_questions_fullclean.csv', encoding='utf-8')
tags_list.to_csv('tags_list.csv', encoding='utf-8')

         Id                                 Title  \
0  10679131          change order array dimension   
1   1169714         difference r program language   
2   8096313                 bind note r cmd check   
3    526457            form fail validation field   
4  10437442             place border around point   
5   2547447    state unit test support r language   
6    487971   way list name python module package   
7   2492947                   boxplot r show mean   
8   7128413  terminology label level factor weird   
9    455552                 break exception pydev   

                                                Body  \
0  reorder dimension n array example three array ...   
1                                       difference r   
2  check package obtain note bind use function li...   
3  model define class articlemodelsmodel slug mod...   
4  would like place border around point scatterpl...   
5  r statistic program language part r use packag...   
6  way list name modules